In [1]:
import os
import pandas as pd
from scipy import stats
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso
from sklearn.metrics import accuracy_score
import math
from numpy import mean
from numpy import std
from numpy import absolute
from numpy import cov
import numpy as np
import inspect
from sklearn.linear_model import LogisticRegression
import glob
from random import *
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_curve
from google.colab import drive
#drive.mount('/content/drive')
#%cd drive/MyDrive/Project271B

In [2]:
df = pd.read_csv('magic_data.csv')
print(df.shape)
print(df.columns[df.isna().any()].tolist())
na_cols = df.columns[df.isna().any()].tolist()

FileNotFoundError: ignored

**Preprocessing**

In [ ]:
df = df.drop(columns=['glucose'])

In [ ]:
df.isna().sum()

male                 0
age                  0
education          105
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
TenYearCHD           0
dtype: int64

In [ ]:
indices_to_drop = []
for index,row in df.iterrows():
  if row.isna().sum()>0:
    indices_to_drop.append(index)

In [ ]:
df2 = df.drop(indices_to_drop)

In [ ]:
df2.isna().sum()

male               0
age                0
education          0
currentSmoker      0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
TenYearCHD         0
dtype: int64

In [ ]:
df2.shape

(3987, 15)

In [ ]:
y = df2['TenYearCHD']
X = df2
X = df2.drop('TenYearCHD', axis=1)

In [ ]:
from sklearn.model_selection import train_test_split 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 0)

In [ ]:
import xgboost as xgb

In [ ]:
X_train = X_train.loc[:, ~X_train.columns.str.contains('^Unnamed')]
X_test = X_test.loc[:, ~X_test.columns.str.contains('^Unnamed')]

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(k_neighbors=15, random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)
OS_data = pd.concat([X_res, y_res], axis=1)

In [ ]:
OS_data.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,TenYearCHD
0,0,38,2.0,0,0.0,0.0,0,0,0,169.0,115.0,60.0,26.87,65.0,0
1,1,40,2.0,1,30.0,0.0,0,1,0,282.0,130.0,80.0,23.90,66.0,0
2,1,49,2.0,0,0.0,0.0,0,1,0,271.0,140.0,108.0,27.66,82.0,0
3,1,38,1.0,1,9.0,0.0,0,0,0,305.0,114.0,80.0,28.61,75.0,0
4,0,63,1.0,1,10.0,1.0,0,1,0,213.0,182.0,92.0,26.87,52.0,0


In [ ]:
OS_data.to_csv('/Users/pragnaelavarthi/Downloads/OS_data')

In [ ]:
OS_test = pd.concat([X_test, y_test], axis=1)
OS_test.to_csv('/Users/pragnaelavarthi/Downloads/OS_test')

**Decision tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dTree_clf = DecisionTreeClassifier(max_depth= 6)

In [ ]:
dTree_clf.fit(X_res,y_res)

DecisionTreeClassifier(max_depth=6)

In [ ]:
y_pred2 = dTree_clf.predict(X_test)

In [ ]:
print("Accuracy of Model::",accuracy_score(y_test,y_pred2))

Accuracy of Model:: 0.7495826377295493


In [ ]:
from sklearn.metrics import roc_auc_score
auc_score = roc_auc_score(y_test,y_pred2)
print(auc_score)

0.6086060883497367


**XGBoost**

In [ ]:
xgb_classifier = xgb.XGBClassifier(max_depth = 5, learning_rate=0.04, booster = 'dart')

In [ ]:
xgb_classifier.fit(X_res,y_res)

XGBClassifier(base_score=None, booster='dart', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.04, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [ ]:
predictions = xgb_classifier.predict(X_test)

In [ ]:
print("Accuracy of Model::",accuracy_score(y_test,predictions))

Accuracy of Model:: 0.8180300500834724


In [ ]:
auc_score = roc_auc_score(y_test,predictions)

In [ ]:
print(auc_score)

0.579755092698558


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
prec = precision_recall_fscore_support(y_test,predictions)
prec = pd.DataFrame(prec)
print(prec)

            0          1
0    0.879925   0.318182
1    0.912451   0.247059
2    0.895893   0.278146
3  514.000000  85.000000


**Handling Imbalance**

In [ ]:
from sklearn.utils.class_weight import compute_sample_weight
sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=y_res
)

xgb_classifier.fit(X_res, y_res, sample_weight=sample_weights)

XGBClassifier(base_score=None, booster='dart', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.04, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [ ]:
predictions_new = xgb_classifier.predict(X_test)

In [ ]:
print("Accuracy of Model::",accuracy_score(y_test,predictions_new))

Accuracy of Model:: 0.8180300500834724


In [ ]:
auc_score = roc_auc_score(y_test,predictions_new)

In [ ]:
print(auc_score)

0.579755092698558


**Adaboost**

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
clf_ada = AdaBoostClassifier(n_estimators=100, random_state=0)

In [ ]:
clf_ada.fit(X_res,y_res)

AdaBoostClassifier(n_estimators=100, random_state=0)

In [ ]:
ada_pred = clf_ada.predict(X_test)

In [ ]:
print("Accuracy of Model::",accuracy_score(y_test,ada_pred))

Accuracy of Model:: 0.7829716193656093


In [ ]:
auc_score = roc_auc_score(y_test,ada_pred)

In [ ]:
print(auc_score)

0.5936942092011902
